In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Загрузка данных
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

def prepare_data(df):
    # Выбираем только числовые столбцы
    numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
    df = df[numeric_features]
    
    # Удаляем 'Id' и 'SalePrice' (если есть) из признаков
    features = df.columns.tolist()
    if 'Id' in features:
        features.remove('Id')
    if 'SalePrice' in features:
        features.remove('SalePrice')
    
    return df[features]

def create_submission(predictions, test_ids, filename='submission.csv'):
    submission = pd.DataFrame({
        'Id': test_ids,
        'SalePrice': predictions
    })
    submission.to_csv(filename, index=False)
    print(f"Файл {filename} создан")
    return submission

# Подготовка данных
X_train = prepare_data(train_data)
y_train = train_data['SalePrice']
X_test = prepare_data(test_data)

# Заполнение пропущенных значений
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Обучение модели
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Получение предсказаний
predictions = model.predict(X_test_scaled)

# Создание файла с предсказаниями
submission = create_submission(predictions, test_data['Id'])

# Вывод информации о модели
print("\nКоэффициент детерминации (R²):", 
      model.score(X_train_scaled, y_train))

# Вывод важности признаков
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': abs(model.coef_)
})
print("\nТоп-10 важных признаков:")
print(feature_importance.sort_values('importance', ascending=False).head(10))

# Базовая статистика предсказаний
print("\nСтатистика предсказаний:")
print(pd.Series(predictions).describe())

Файл submission.csv создан

Коэффициент детерминации (R²): 0.8131246345519315

Топ-10 важных признаков:
         feature    importance
3    OverallQual  23950.605009
15     GrLivArea  14193.847098
13      2ndFlrSF   9588.472271
12      1stFlrSF   8474.787428
25    GarageCars   8392.940096
20  BedroomAbvGr   8233.270092
22  TotRmsAbvGrd   8227.218959
5      YearBuilt   8201.819759
0     MSSubClass   7678.914562
7     MasVnrArea   5679.467848

Статистика предсказаний:
count      1459.000000
mean     177923.193934
std       71681.518957
min        1453.858214
25%      127941.278619
50%      168006.291047
75%      222617.264808
max      634895.074709
dtype: float64
